In [20]:
# Use the native inference API to send a text message to Meta Llama 3.

import boto3
import json

from botocore.exceptions import ClientError

# Create a Bedrock Runtime client in the AWS Region of your choice.
client = boto3.client("bedrock-runtime", region_name="us-west-2")

# Set the model ID, e.g., Llama 3 70b Instruct.
model_id = "us.meta.llama3-2-11b-instruct-v1:0"

# Define the prompt for the model.
prompt = "Describe the purpose of a 'hello world' program in one line."

# Embed the prompt in Llama 3's instruction format.
formatted_prompt = f"""
<|begin_of_text|><|start_header_id|>user<|end_header_id|>
{prompt}
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""

# Format the request payload using the model's native structure.
native_request = {
    "prompt": formatted_prompt,
    "max_gen_len": 512,
    "temperature": 0.5,
}

# Convert the native request to JSON.
request = json.dumps(native_request)

try:
    # Invoke the model with the request.
    response = client.invoke_model(modelId=model_id, body=request)

except (ClientError, Exception) as e:
    print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
    exit(1)

# Decode the response body.
model_response = json.loads(response["body"].read())

# Extract and print the response text.
response_text = model_response["generation"]
print(response_text)




A "Hello, World!" program is a simple program that displays the text "Hello, World!" on the screen, serving as a basic example to introduce programming concepts and test the functionality of a programming environment.


In [113]:
filename = "temp.webp"
with open(f"./img/{filename}", "rb") as image_file:
    image_bytes = image_file.read()

In [114]:
def UserMessage(text, image_bytes=None, image_format=None):
    content = [{"text": text}]
    if image_bytes:
        image = {"image": {"format": image_format, "source": {"bytes": image_bytes}}}
        content.append(image)
    return {"role": "user", "content": content}


def AIMessage(text):
    return {"role": "assistant", "content": [{"text": text}]}


def extract_response(response):
    return response["output"]["message"]["content"][0]["text"]

In [120]:
messages = [UserMessage(text="What is in the image?", image_bytes=image_bytes, image_format="webp")]

response = client.converse(
    modelId=model_id,
    messages=messages
)

In [121]:
messages.append(AIMessage(text=response["output"]["message"]["content"][0]["text"]))
# this one yes yes: b/c image must be add only once in the chat history, so add them only the last message
messages.append(UserMessage(text="How many of them?"))
# this one no no
# messages.append(UserMessage(text="How many color do you see?", image_bytes=image_bytes, image_format="png"))

In [122]:
response = client.converse(
    modelId=model_id,
    messages=messages
)

In [123]:
extract_response(response)

'There is one piglet in the image.'